In [ ]:
import scanpy as sc
import numpy as np
import pandas as pd
import os
import gc
import anndata as ad
#import scarches as sca
#import pytorch_lightning as pl
#from scarches.models.scpoli import scPoli
#from sklearn.metrics import classification_report
import sys
import traceback
import matplotlib.pyplot as plt
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

In [ ]:
import os
os.chdir('/home/aih/shrey.parikh/PDAC/PDAC/processed_datasets/')

In [ ]:
adata_filtered = sc.read_h5ad('All_genes/Concat_All_Genes_filtered.h5ad')

In [ ]:
adata_sc = adata_filtered[adata_filtered.obs.Condition == 'scRNA-seq'].copy()

In [ ]:
sc.pp.highly_variable_genes(adata_sc, layer='log_norm', batch_key='batch_covariate')
adata_hvg_sc = adata_sc[:, adata_sc.var.highly_variable].copy()

In [ ]:
del adata_filtered
gc.collect()

# integration performed with script unsupervised_scpoli.py

In [ ]:
adata_int = sc.read_h5ad('../../PDAC_Final/single_cell_int/adata_unsup_scRNA.h5ad')

In [ ]:
adata_hvg_sc.write('../../PDAC_Final/single_cell_int/adata_hvg_sc.h5ad')

In [ ]:
adata_sc.obsm['X_scpoli'] = adata_int.X.copy()

In [ ]:
adata_sc.obsm['X_umap_int'] = adata_int.obsm['X_umap'].copy()

In [ ]:
adata_sc.obsp = adata_int.obsp.copy()
adata_sc.uns = adata_int.uns.copy()

In [ ]:
sc.pl.umap(adata_int, color=['batch_covariate' , 'leiden'], frameon=False, wspace=0.5)

In [ ]:
adata_sc.obs['leiden'] = adata_sc.obs.Dataset_Barcode.map(dict(zip(adata_int.obs.Dataset_Barcode, adata_int.obs['leiden'])))

In [ ]:
adata_sc.obsm['X_umap'] = adata_sc.obsm['X_umap_int']

In [ ]:
sc.pl.umap(adata_sc, color=['batch_covariate' , 'leiden'], frameon=False, wspace=0.5)

In [ ]:
adata_sc.write('../../PDAC_Final/single_cell_int/adata_sc_int.h5ad')

# Reread

In [ ]:
adata_sc = sc.read_h5ad('../../PDAC_Final/single_cell_int/adata_sc_int.h5ad')
# TODO: necessary?

# Annotate the object

In [ ]:
sc.pp.pca(adata_sc)
sc.tl.leiden(adata_sc, resolution=0.2, key_added='leiden_0.2')

In [ ]:
sc.pl.umap(adata_sc, color=['batch_covariate' , 'leiden_0.2'], frameon=False, wspace=0.5)

In [ ]:
sc.pl.umap(adata_sc, color=["ACTA2", "PDGFRB", "COL1A1", "PECAM1", "VWF"], frameon=False, layer='log_norm', vmax=5)

In [ ]:
markers = {
    "Malignant": ["KRT19", "SOX9", "MUC1", ],
    "Epithelial": ["EPCAM", "KRT19", "KRT7", "CFTR"],
    "Immune": ["CD3D", "CD4", "CD8A", "NCAM1", "MS4A1", "CD68", "CD14", 'PTPRC'],
    "Stromal": ["ACTA2", "PDGFRB", "COL1A1", "PECAM1", "VWF"],
    "Neuronal": ["MAP2", "PAX6", "NEUROD1", "MAPT"],
    "Endocrine": ['INS', 'SST', "GCG", "GHRL"]
}
sc.pl.matrixplot(adata_sc, markers, groupby="leiden_0.2", cmap="viridis", standard_scale='var', dendrogram=True, swap_axes=False)

In [ ]:
marker = []
for celltype,marker in markers.items():
    # marker.append('leiden_0.2')
    sc.pl.umap(adata_sc, color=marker, frameon=False, layer='log_norm', ncols=2, legend_loc='on data', size=1.5)

In [ ]:
for leiden_cluster in adata_sc.obs['leiden_0.25'].unique().tolist():
    adata_temp = adata_sc[adata.obs['leiden_0.25'] == leiden_cluster]
    for celltype,marker in markers.items():
        # marker.append('leiden_0.2')
        
        sc.pl.umap(adata_temp, color=marker, frameon=False, layer='log_norm', ncols=2, legend_loc='on data', size=1.5)

In [ ]:
cluster 1, 4, 10, 11: epithelial/malignant
cluster 0, 2, 6: immune 
cluster 3, 5, 7: stromal 
cluster 14: seems endorcine

cluster 10 seems ambiguous (KRT19, KRT7 but also CD68 and PTPRC?)

#TODO: remove?


In [ ]:
markers = ["KRT19", "EPCAM", "PTPRC", "CD3D", "ACTA2", "PDGFRB", "MAP2", "PAX6", "INS", "SST", "leiden_0.2"]


In [ ]:
unique_clusters = list(map(str, (sorted(list(map(int, unique_clusters))))))

In [ ]:
for leiden_cluster in sorted(unique_clusters):
    # fig, axs = plt.subplots(1, len(markers), figsize=(20, 5))
    adata_temp = adata_sc[adata_sc.obs[leiden_res] == leiden_cluster]
    sc.pl.umap(
        adata_temp, color=markers, frameon=False, layer='log_norm', ncols=6,
        show=False, size=1.5, legend_loc='on data')

    plt.tight_layout()
    plt.show()

cluster 1, 4, 9, 11: epithelial/malignant
cluster 0, 2, 6: immune 
cluster 3, 7: stromal 
cluster 14, 15: neuronal

cluster 4: half has epithelial/malignant but other half has expression of none
cluster 5: some expression of INS and KRT19
cluster 8: some expression of PTPRC but otherwise no sig exp
cluster 10: seems epithelial but ambiguous (KRT19, KRT7 but also CD68 and PTPRC?)
cluster 12: none of the marker expression
cluster 13: divided between endocrine and neuronal


In [ ]:
sc.tl.rank_genes_groups(adata_sc, groupby='leiden_0.2', layer='log_norm')
sc.pl.rank_genes_groups(adata_sc)

cluster 1, 4, 9, 11: epithelial/malignant
cluster 0, 2, 6: immune 
cluster 3, 7: stromal 
cluster 14: seems endorcine
cluster 15: neuronal

cluster 4: half has epithelial/malignant but other half has expression of none
cluster 5: some expression of INS and KRT19
cluster 8: some expression of PTPRC but otherwise no sig exp
cluster 10: seems epithelial but ambiguous (KRT19, KRT7 but also CD68 and PTPRC?)
cluster 12: none of the marker expression
cluster 13: divided between endocrine and neuronal


In [ ]:
df_de_adata_sc = pd.DataFrame(adata_sc.uns['rank_genes_groups']['names'])

In [ ]:
cluster_mapping = {
    '1': 'epithelial/malignant',
    '9': 'epithelial/malignant',
    '11': 'epithelial/malignant',
    '0': 'immune',
    '2': 'immune',
    '6': 'immune',
    '3': 'stromal',
    '7': 'stromal',
    '15': 'neuronal'
}


In [ ]:
adata_sc.obs['Level_0'] = adata_sc.obs['leiden_0.2'].map(cluster_mapping).fillna('Unknown')


# 4,5,8,10,12,13 seems ambiguous
# subcluster and see what those are to better reveal the cell type

# Leiden 4

In [ ]:
def analyse_subclusters(adata, leiden_cluster, resolution=0.1):
    subcluster = adata_sc[adata_sc.obs['leiden_0.2'] == leiden_cluster]
    sc.pp.neighbors(subcluster, use_rep='X_scpoli')
    sc.tl.leiden(subcluster, resolution=resolution)
    sc.pl.umap(subcluster, color=['leiden', 'leiden_0.2', "KRT19", "EPCAM", "PTPRC", "CD3D", "ACTA2", "PDGFRB", "MAP2", "PAX6", "INS", "SST"], frameon=False, layer='log_norm')
    sc.tl.rank_genes_groups(subcluster, groupby='leiden', layer='log_norm')
    sc.pl.rank_genes_groups(subcluster)
    df = pd.DataFrame(subcluster.uns['rank_genes_groups']['names'])
    subcluster.obs['subcluster'] = [leiden_cluster + '_' + x for x in subcluster.obs['leiden']]
    dict_map = dict(list(zip(subcluster.obs.Dataset_Barcode, subcluster.obs['subcluster'])))
    if 'leiden_subcluster' not in adata_sc.obs.columns:
        adata_sc.obs['leiden_subcluster'] = 'Unknown'
    adata_sc.obs['leiden_subcluster'] = adata_sc.obs['Dataset_Barcode'].map(dict_map).fillna(adata_sc.obs['leiden_subcluster'])
    return df, subcluster

In [ ]:
df_4, leiden_4 = analyse_subclusters(adata_sc, '4')

subcluster 0,1,2: epithelial (level_0, level_1: ductal, acinar, acinar(REG1))
subcluster 3: stromal (level_1: stellate)

In [ ]:
{'4_0':'epithelial/malignant','4_1':'epithelial/malignant', '4_2': 'epithelial/malignant', '4_3':'stromal'}

# Leiden_5

In [ ]:
df_5, leiden_5 = analyse_subclusters(adata_sc, '5')

level_1
1: mix of acinar/endocrine
2: pericyte/vascular smooth muscle cells 
4: ductal/acinar
5: macrophages/erythroid lineage.

In [ ]:

{
    '5_0': 'stromal',
    '5_1': 'Ambiguous',
    '5_2': 'stromal',
    '5_3': 'epithelial/malignant',
    '5_4': 'epithelial/malignant',
    '5_5': 'Ambiguous'
}

# Leiden 8

In [ ]:
df_8, leiden_8 = analyse_subclusters(adata_sc, '8')

level_1
'8_0': mast cells,
'8_1': ductal epithelial,
'8_2': T cells,
'8_3': macrophages/neutrophils),
'8_4': acinar cells,
'8_5': fibroblasts,
'8_6': plasma cells

In [ ]:
{
    '8_0': 'immune',
    '8_1': 'epithelial/malignant',
    '8_2': 'immune',
    '8_3': 'immune',
    '8_4': 'epithelial/malignant',
    '8_5': 'stromal',
    '8_6': 'immune'
}

# Leiden 10

In [ ]:
df_10, leiden_10 = analyse_subclusters(adata_sc, '10')

## 9_0 is still ambiguous

In [ ]:
{
    '10_0': 'stromal',
    '10_1': 'epithelial/malignant',
    '10_2': 'erythroid'
}

# Leiden 12

In [ ]:
df_12, leiden_12 = analyse_subclusters(adata_sc, '12')

level_1
0: B cells/plasma cells
1: 
2: acinar cells
3: ductal
4: ambiguous but immune
5: fibroblasts

In [ ]:
{
    '12_0': 'immune',
    '12_1': 'Ambiguous',
    '12_2': 'epithelial/malignant',
    '12_3': 'epithelial/malignant',
    '12_4': 'immune',
    '12_5': 'stromal'
}

# Leiden 13

In [ ]:
df_13, leiden_13 = analyse_subclusters(adata_sc, '13')

level_1
1: Ambiguous
2: islets cells
3: Ambiguous
4: endocrine but mixed with acinar
5: ductal

In [ ]:
{
    '13_0': 'Ambiguous',
    '13_1': 'endocrine',
    '13_2': 'Ambiguous',
    '13_3': 'endocrine',
    '13_4': 'epithelial'
}

# Leiden 14

In [ ]:
df_14, leiden_14 = analyse_subclusters(adata_sc, '14', resolution=0.2)

'14_0': 'stromal/immune',
'14_1': 'stromal/neuronal',
'14_2': 'stromal/immune'

In [ ]:
{
    '14_0': 'Ambiguous',
    '14_1': 'Ambiguous',
    '14_2': 'Ambiguous'
}

In [ ]:
subcluster_dict  = {'4_0':'epithelial/malignant','4_1':'epithelial/malignant', '4_2': 'epithelial/malignant', '4_3':'stromal',
                    '5_0': 'stromal', '5_1': 'Ambiguous', '5_2': 'stromal', '5_3': 'epithelial/malignant', '5_4': 'epithelial/malignant', '5_5': 'Ambiguous',
                    '8_0': 'immune', '8_1': 'epithelial/malignant', '8_2': 'immune', '8_3': 'immune', '8_4': 'epithelial/malignant', '8_5': 'stromal', '8_6': 'immune',
                    '10_0': 'stromal', '10_1': 'epithelial/malignant', '10_2': 'erythroid',
                    '12_0': 'immune', '12_1': 'Ambiguous', '12_2': 'epithelial/malignant', '12_3': 'epithelial/malignant', '12_4': 'immune', '12_5': 'stromal',
                    '13_0': 'Ambiguous', '13_1': 'endocrine', '13_2': 'Ambiguous', '13_3': 'endocrine', '13_4': 'epithelial/malignant',
                    '14_0': 'Ambiguous', '14_1': 'Ambiguous', '14_2': 'Ambiguous'
                    }

In [ ]:
adata_sc.obs['Level_0'] = adata_sc.obs['leiden_0.2'].map(cluster_mapping).fillna('Unknown')

In [ ]:
adata_sc.obs['Level_0'] = adata_sc.obs['leiden_subcluster'].map(subcluster_dict).fillna(adata_sc.obs['Level_0'])

In [ ]:
cell_type_mapping = {
    'endocrine': 'Endocrine Cell',
    'epithelial/malignant': 'Epithelial/Malignant Cell',
    'erythroid': 'Erythroid Cell',
    'immune': 'Immune Cell',
    'neuronal': 'Neuronal Cell',
    'stromal': 'Stromal Cell',
    'Ambiguous': 'Ambiguous'
}
adata_sc.obs['Level_0'] = adata_sc.obs['Level_0'].map(cell_type_mapping) 

In [ ]:
sc.pl.umap(adata_sc, color=['Dataset', 'Level_0', 'leiden_0.2'], frameon=False)

In [ ]:
# from the previous analysis
'4_0':'Ductal Cell',
'4_1': 'Acinar Cell',
'4_2': 'Acinar Cell'
'4_3': 'Stellate Cell'
'5_1': 'mix of acinar/endocrine
'5_2': pericyte/vascular smooth muscle cells 
'5_4': ductal/acinar
'5_5': macrophages/erythroid lineage.
subcluster 3: stromal (level_1: stellate)
#TODO: remove?

# Deep dive into ambiguous

In [ ]:
def analyse_subclusters(adata, cell_type, resolution=0.1):
    subcluster = adata_sc[adata_sc.obs['Level_0'] == cell_type]
    sc.pp.neighbors(subcluster, use_rep='X_scpoli')
    sc.tl.leiden(subcluster, resolution=resolution)
    sc.pl.umap(subcluster, color=['leiden', 'leiden_0.2'], frameon=False, layer='log_norm')
    sc.tl.rank_genes_groups(subcluster, groupby='leiden', layer='log_norm')
    sc.pl.rank_genes_groups(subcluster)
    df = pd.DataFrame(subcluster.uns['rank_genes_groups']['names'])
    subcluster.obs['subcluster'] = [cell_type + '_' + x for x in subcluster.obs['leiden']]
    dict_map = dict(list(zip(subcluster.obs.Dataset_Barcode, subcluster.obs['subcluster'])))
    if 'level0_leiden_subcluster' not in adata_sc.obs.columns:
        adata_sc.obs['level0_leiden_subcluster'] = 'Unknown'
    adata_sc.obs['level0_leiden_subcluster'] = adata_sc.obs['Dataset_Barcode'].map(dict_map).fillna(adata_sc.obs['level0_leiden_subcluster'])
    return df, subcluster

In [ ]:
ambi = adata_sc[adata_sc.obs.Level_0 == 'Ambiguous']

In [ ]:
sc.pl.umap(ambi, color='Level_0', frameon=False)

In [ ]:
df_ambi, ambi = analyse_subclusters(adata_sc, 'Ambiguous', resolution=0.5)

In [ ]:
sc.tl.rank_genes_groups(ambi, groupby='leiden_0.2', groups=
#TODO: fix—what groups?

'Ambiguous_0': 'Endothelial Cell',
'Ambiguous_1': 'Neuronal Cell',
'Ambiguous_2': 'Endothelial Cell',
'Ambiguous_3': 'Epithelial/Malignant Cell',
'Ambiguous_4':'Epithelial/Malignant Cell',
'Ambiguous_5': 'Immune Cell',
'Ambiguous_6': 'Epithelial/Malignant Cell',
'Ambiguous_7': 'Stromal Cell' (adipocytes)
'Ambiguous_8': 'Erythroid Cell;
'Ambiguous_9': 'Epithelial/Malignant Cell'

In [ ]:
dict_ambi = {'Ambiguous_0': 'Endothelial Cell',
'Ambiguous_1': 'Neuronal Cell',
'Ambiguous_2': 'Endothelial Cell',
'Ambiguous_3': 'Epithelial/Malignant Cell',
'Ambiguous_4':'Epithelial/Malignant Cell',
'Ambiguous_5': 'Immune Cell',
'Ambiguous_6': 'Epithelial/Malignant Cell',
'Ambiguous_7': 'Stromal Cell',
'Ambiguous_8': 'Erythroid Cell',
'Ambiguous_9': 'Epithelial/Malignant Cell'}

In [ ]:
adata_sc.obs.Level_0 = adata_sc.obs.level0_leiden_subcluster.map(dict_ambi).fillna(adata_sc.obs.Level_0)

In [ ]:
sc.pl.umap(adata_sc, color=['Dataset', 'Level_0', 'leiden_0.2'], frameon=False)

# Level 1 Annotation

# Immune

In [ ]:
df_immune, immune = analyse_subclusters(adata_sc, 'Immune Cell', resolution=0.2)

In [ ]:
t_cells = ['CD4', 'CD3D', 'THEMIS', 'CD8A', 'CD8B']
t_cells.append('leiden')

In [ ]:
sc.pl.umap(immune, color=t_cells, frameon=False, layer='log_norm')

Level_2
'immune_0': 'CD8 Cell',
'immune_1': 'CD4 Cell',
'immune_5': 'Mast Cell',
'immune_6': 'Plasma Cell',


In [ ]:
{
    'immune_0': 'T Cell',
    'immune_1': 'T Cell',
    'immune_2': 'NK Cell',
    'immune_3': 'B Cell',
    'immune_4': 'Myeloid Cell',
    'immune_5': 'Myeloid Cell',
    'immune_6': 'B Cell',
    'immune_7': 'Erythroid Cell',
    'immune_8': 'Ambiguous_Immune',
    'immune_9': 'Myeloid Cell',
    'immune_10': 'Ambiguous_Immune'
}

# Stromal

In [ ]:
df_stromal, stromal = analyse_subclusters(adata_sc, 'Stromal Cell', resolution=0.2)

Level_2
stromal_2: ACTA2, TAGLN, TPM2, CALD1,  myCAF
'stromal_4': 'macrophages/inflammatory stromal cells',
'stromal_8': macrophage/dendritic cells
'stromal_9': 'mast cells'
'stromal_10': 'mast cells'

In [ ]:
{
    'stromal_0': 'Fibroblast',
    'stromal_1': 'Fibroblast',
    'stromal_2': 'Fibroblast',
    'stromal_3': 'Endothelial Cell',
    'stromal_4': 'Ambiguous_Stromal',
    'stromal_5': 'Fibroblast',
    'stromal_6': 'Adipocyte',
    'stromal_7': 'Acinar Cell',
    'stromal_8': 'Adipocyte',
    'stromal_9': 'Myeloid Cell',
    
}

In [ ]:
adata_sc.obs.rename(columns={'level0_leidein_subcluster': 'level0_leiden_subcluster'}, inplace=True)

# Epithelial

In [ ]:
df_epi, epi = analyse_subclusters(adata_sc, 'Epithelial/Malignant Cell', resolution=0.2)

In [ ]:
markers_csv = pd.read_csv('../../PDAC_Final/single_cell_int/markers.csv', index_col='Unnamed: 0')

In [ ]:
malignant_markers = list(set(markers_csv.iloc[18:21, :].values.flatten()))

In [ ]:
count_per_column = {}
for col in df_epi:
    count = df_epi[col][:100].isin(malignant_markers).sum()
    count_per_column[col] = count

In [ ]:
pd.DataFrame(list(count_per_column.items()), columns=['Cluster', 'Malignant Marker Count']).sort_values(by='Malignant Marker Count', ascending=False)

mal/non mal classification
'epithelial_0': 'Ductal Cell/Malignant' (KRT19, CEACAM6, S100A)
'epithelial_1': 'Ductal Cell/Malignant ', (TFF1, MUC1)
'epithelial_2': 'Ductal Cell',
'epithelial_3': 'Ductal Cell'
'epithelial_4': 'Acinar Cell',
'epithelial_5': 'Ductal Cell/ Atypical', (REG1A, TFF2)
'epithelial_6': 'Acinar Cell' (REG1B),
'epithelial_7': 'Ambiguous',
'epithelial_8': 'potentially EMT' 
'epithelial_9': 'potentially EMT/ proliferating Malignant cells', (S100, CEACAM5
'epithelial_10': 'potentially EMT' 
'epithelial_11': 'Myeloid Cell' Mast Cells
'epithelial_12': 'B Cell'
'epithelial_13': 'high MT, could be contaminants',
'epithelial_14': 'Ductal Cell/Malignant',    

In [ ]:
{
    'epithelial_0': 'Ductal Cell',
    'epithelial_1': 'Ductal Cell',
    'epithelial_2': 'Ductal Cell',
    'epithelial_3': 'Ductal Cell',
    'epithelial_4': 'Acinar Cell',
    'epithelial_5': 'Ductal Cell',
    'epithelial_6': 'Acinar Cell',
    'epithelial_7': 'Acinar Cell',
    'epithelial_8': 'EMT',
    'epithelial_9': 'EMT',
    'epithelial_10': 'EMT',
    'epithelial_11': 'Myeloid Cell',
    'epithelial_12': 'B Cell',
    'epithelial_13': 'Ambiguous_Epithelial',
    'epithelial_14': 'Ductal Cell',    
}

# Endocrine 

In [ ]:
df_endocrine, endocrine = analyse_subclusters(adata_sc, 'Endocrine Cell', resolution=0.2)

In [ ]:
df_endocrine.head(10)

In [ ]:
{'endo_0': '

#TODO: ???

# Mapping Level_1

In [ ]:
temp = {    'immune_0': 'T Cell',
    'immune_1': 'T Cell',
    'immune_2': 'NK Cell',
    'immune_3': 'B Cell',
    'immune_4': 'Myeloid Cell',
    'immune_5': 'Myeloid Cell',
    'immune_6': 'B Cell',
    'immune_7': 'Erythroid Cell',
    'immune_8': 'Ambiguous_Immune',
    'immune_9': 'Myeloid Cell',
    'immune_10': 'Ambiguous_Immune',
    'stromal_0': 'Fibroblast',
    'stromal_1': 'Fibroblast',
    'stromal_2': 'Fibroblast',
    'stromal_3': 'Endothelial Cell',
    'stromal_4': 'Ambiguous_Stromal',
    'stromal_5': 'Fibroblast',
    'stromal_6': 'Adipocyte',
    'stromal_7': 'Acinar Cell',
    'stromal_8': 'Adipocyte',
    'stromal_9': 'Myeloid Cell',
    'epithelial_0': 'Ductal Cell/Malignant',
    'epithelial_1': 'Ductal Cell/Malignant',
    'epithelial_2': 'Ductal Cell',
    'epithelial_3': 'Ductal Cell',
    'epithelial_4': 'Acinar Cell',
    'epithelial_5': 'Ductal Cell',
    'epithelial_6': 'Acinar Cell',
    'epithelial_7': 'Acinar Cell',
    'epithelial_8': 'EMT',
    'epithelial_9': 'EMT',
    'epithelial_10': 'EMT',
    'epithelial_11': 'Myeloid Cell',
    'epithelial_12': 'B Cell',
    'epithelial_13': 'Ambiguous_Epithelial',
    'epithelial_14': 'Ductal Cell/Malignant'}
celltype_subcluster = {key.replace('immune_', 'Immune Cell_').replace('stromal_', 'Stromal Cell_').replace('epithelial_', 'Epithelial/Malignant Cell_'): value
                       for key, value in temp.items()}


In [ ]:
adata_sc.obs['Level_1'] = adata_sc.obs['Level_0']
adata_sc.obs['Level_1'] = adata_sc.obs['level0_leiden_subcluster'].map(celltype_subcluster).fillna(adata_sc.obs['Level_0'])

In [ ]:
sc.tl.leiden(adata_sc, resolution=0.5, key_added='leiden_0.5')

In [ ]:
sc.pl.umap(adata_sc, color=['Dataset', 'Level_0', 'level0_leiden_subcluster', 'Level_1'], frameon=False, ncols=1, size=1, wspace=0.3)

In [ ]:
adata_sc.write('../../PDAC_Final/single_cell_int/adata_sc_int.h5ad')